<a href="https://colab.research.google.com/github/ulubeykhuja/mohir_dev_lessons/blob/main/ML_boshlanishi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4-qadam. Machine Learning**

In [3]:
import pandas as pd
import numpy as np
import sklearn

# Onlayn datasetni ochish
URL = "https://raw.githubusercontent.com/ageron/handson-ml2/refs/heads/master/datasets/housing/housing.csv"
df = pd.read_csv(URL)
df.head()

# trein va test ga bo'lish
train_set, test_set = train_test_split(df,test_size=0.2, random_state = 42)
# train uchun bashorat qilinadigan qator (median_house_value)ni tashlab yuborish
X_train = train_set.drop("median_house_value", axis=1)
# bashorat qilinadigan qator (median_house_value)ni alohida saqlash
y = train_set["median_house_value"].copy()

# son bo'magan qatorni tashlash
X_num = X_train.drop("ocean_proximity", axis=1)

In [6]:
# Transformer yasash
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [9]:
# Konveyr yasash uchun library
from sklearn.pipeline import Pipeline
# malumotlar bir nechta jarayonlardan otdi
# kerakli lib larni chaqiraiz
from sklearn.impute import SimpleImputer # medianlar uchun class
from sklearn.preprocessing import OneHotEncoder, StandardScaler #

# Sonli ustunlar uchun alohida konveyr yasash
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# chaqirish
num_pipeline.fit_transform(X_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

In [ ]:
#housing_onehot = pd.get_dummies(housing['ocean_proximity'])
# housing_onehot

In [12]:
# Matnli konveyr yasash
from sklearn.compose import ColumnTransformer
num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

#Katta konveyr
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs), # num_pipeline ni full konveyrga solish
    # 3qismga bo'lindi, ustunlar ajratildi
    ("cat", OneHotEncoder(), cat_attribs)
])

In [13]:
full_pipeline.fit_transform(X_train)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [19]:
X_prepared = full_pipeline.fit_transform(X_train)

In [21]:
X_prepared

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

havola : https://colab.research.google.com/github/MohirdevTexnik/MachineLearning_Noutbook/blob/main/05-ml-03-Pipeline.ipynb

# **Liner Regression**

In [22]:
# Liner regressiya lib ni import qilish
from sklearn.linear_model import LinearRegression

# obyektdsn nusxa olish
LR_model = LinearRegression()

In [24]:
# LR modelni yasash
LR_model.fit(X_prepared, y)

LinearRegression()

In [26]:
# tekshirish
# boshqa qiymatlarni bashorat qilish

test_data = X_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
10085,-117.95,33.94,28.0,2851.0,496.0,1287.0,496.0,5.0782,<1H OCEAN
6725,-118.11,34.12,48.0,2476.0,313.0,900.0,303.0,10.6767,<1H OCEAN
11521,-118.06,33.73,16.0,4392.0,602.0,1490.0,578.0,10.5424,<1H OCEAN
5479,-118.45,34.00,48.0,1923.0,408.0,1142.0,433.0,4.5750,<1H OCEAN
14078,-117.11,32.77,48.0,1502.0,272.0,590.0,265.0,2.5952,NEAR OCEAN
19445,-121.02,37.69,19.0,3814.0,790.0,2219.0,804.0,3.5208,INLAND
19430,-121.06,37.66,6.0,3655.0,598.0,1993.0,596.0,4.6053,INLAND
2815,-119.05,35.42,35.0,2353.0,483.0,1368.0,455.0,2.3250,INLAND
13251,-117.69,34.11,16.0,2427.0,522.0,794.0,491.0,2.6929,INLAND
73,-122.29,37.81,46.0,12.0,4.0,18.0,7.0,0.4999,NEAR BAY


In [29]:
#tekshirish uchun labellarni ajratib olamiz
test_label = y.loc[test_data.index]
test_label

,median_house_value
10085,264100.0
6725,500001.0
11521,500001.0
5479,326700.0
14078,190300.0
19445,145000.0
19430,150100.0
2815,63200.0
13251,119300.0
73,67500.0


In [33]:
# teat_data ni ham pipeline dan o'tkazamiz

test_data_prepered = full_pipeline.transform(test_data)
test_data_prepered[0:3,]

array([[ 0.81386932, -0.79713037, -0.04826847,  0.09610827, -0.10142582,
        -0.12264759, -0.01046555,  0.62883255,  0.13100512, -0.04337416,
        -0.67048617,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.73409241, -0.7128844 ,  1.53876638, -0.07633875, -0.53818582,
        -0.46301041, -0.51708517,  3.56885639,  1.14622343, -0.0109397 ,
        -1.49058433,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.75902269, -0.89541734, -1.00048937,  0.80475057,  0.15156085,
         0.05588898,  0.20478216,  3.49832943,  0.90620306, -0.04483407,
        -1.30687754,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [34]:
predicted_labels = LR_model.predict(test_data_prepered)
predicted_labels

array([261734.75537741, 501952.26332128, 484527.64847642, 278223.14473638,
       190999.02430281, 146172.3738078 , 154739.54314288, 110274.99924781,
       130689.28838081, 125266.41445987])

In [37]:
# DF ga aylantirib solishtiramiz
pd.DataFrame({'Bashorat': predicted_labels, 'Real_bahosi': test_label, 'farqi': predicted_labels / test_label*100})


,Bashorat,Real_bahosi,farqi
10085,261734.755377,264100.0,99.104413
6725,501952.263321,500001.0,100.390252
11521,484527.648476,500001.0,96.905336
5479,278223.144736,326700.0,85.161660
14078,190999.024303,190300.0,100.367328
19445,146172.373808,145000.0,100.808534
19430,154739.543143,150100.0,103.090968
2815,110274.999248,63200.0,174.485758
13251,130689.288381,119300.0,109.546763
73,125266.414460,67500.0,185.579873


# **5-qadam. Modelni baholash**

In [39]:
X_test = test_set.drop("median_house_value", axis=1)
X_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,INLAND


In [40]:
y_test = test_set['median_house_value'].copy()
y_test

,median_house_value
20046,47700.0
3024,45800.0
15663,500001.0
20484,218600.0
9814,278000.0
...,...
15362,263300.0
16623,266800.0
18086,500001.0
2144,72300.0


In [41]:
X_test_prepered = full_pipeline.transform(X_test)
X_test_prepered

array([[ 0.28534728,  0.1951    , -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.06097472, -0.23549054,  0.11043502, ...,  0.        ,
         0.        ,  0.        ],
       [-1.42487026,  1.00947776,  1.85617335, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.23041404,  0.78014149, -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [-0.08860699,  0.52740357,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60445493, -0.66608108, -0.92113763, ...,  0.        ,
         0.        ,  0.        ]])

In [42]:
y_predicted = LR_model.predict(X_test_prepered)

In [43]:
# modelni baholash
y_predicted

array([ 61874.25460143, 121853.52511139, 267770.94368091, ...,
       447837.04647878, 117275.9214608 , 185597.46125194])

In [48]:
# MAE va R2 ni topish
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)
print ('MAE = ', mae) # xatoni chiqarish

MAE =  50898.73953494079


In [50]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print ('RMSE = ', np.sqrt(mse)) # xatoni chiqarish

RMSE =  72701.32600762135


## **Random Forest**

In [51]:
# bashoratda xatolik kattalifi uchun boshqa algoritmni sibaymiz
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [53]:
y_predicted = RF_model.predict(X_test_prepered)

In [54]:
# buni ham tekshiramiz
mse = mean_squared_error(y_test, y_predicted)
print ('RMSE = ', np.sqrt(mse)) # xatoni chiqarish

RMSE =  50343.26959708822
